In [11]:
SEED = 11

In [2]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from IPython.display import display, Math

In [3]:
import torch.nn as nn
import torch.optim as optim
# import torch.nn.init as init
# from torchvision import datasets
# from torchvision import transforms
# from torch.utils.data import DataLoader

In [4]:
n_hidden = 35
lr = 0.01
epochs = 1000

In [5]:
my_string = 'hello pytorch. how long can a rnn cell remember? i am a boy and you are a girl. can you make a pancake for mommy?'
chars = 'abcdefghijklmnopqrstuvwxyz ?!.,:;01'
char_list = [i for i in chars]
n_letters = len(char_list)

In [6]:
def str2onehot(string):
    start = np.zeros(shape = n_letters, dtype = int)
    end = np.zeros(shape = n_letters, dtype = int)
    start[-2] = 1
    end[-1] = 1
    for c in string:
        idx = char_list.index(c)
        zero = np.zeros(shape = n_letters, dtype = int)
        zero[idx] = 1
        start = np.vstack([start, zero])
    output = np.vstack([start, end])
    return output

def onehot2word(onehot_1):
    onehot = torch.Tensor.numpy(onehot_1)
    return char_list[onehot.argmax()]

In [7]:
class my_RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(my_RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.in2hid = nn.Linear(input_size, hidden_size)
        self.hid2hid = nn.Linear(hidden_size, hidden_size)
        self.hid2out = nn.Linear(hidden_size, output_size)
        self.act_fn = nn.Tanh()
        
    def forward(self, input, hidden):
        hidden = self.act_fn(self.in2hid(input)+self.hid2hid(hidden))
        
#         hidden = self.act_fn(self.hid2hid(hidden)+self.hid2hid(hidden))
        output = self.hid2out(hidden)
        return output, hidden
    
    def init_hidden(self):
        return torch.zeros(1, self.hidden_size)
    

    

In [8]:
rnn = my_RNN(n_letters, n_hidden, n_letters)

In [9]:
loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr = lr)

In [10]:
one_hot = torch.from_numpy(str2onehot(my_string)).type_as(torch.FloatTensor())

In [11]:
for ii in range(epochs):
    rnn.zero_grad()
    total_loss = 0
    hidden = rnn.init_hidden()
    
    ## go all sequence and sum all loss
    for jj in range(one_hot.size()[0]-1):
        input_ = one_hot[jj:jj+1, :]
        target = one_hot[jj+1]
        
        output, hidden = rnn.forward(input_, hidden)
        loss = loss_func(output.view(-1), target.view(-1))
        
        total_loss += loss
        input_ = output
    
    # update weight after calculating all loss through sequence
    total_loss.backward()
    optimizer.step()
    
    if ii%10 == 0:
        print(total_loss)
    
    

tensor(4.9923, grad_fn=<AddBackward0>)
tensor(2.4587, grad_fn=<AddBackward0>)
tensor(1.9323, grad_fn=<AddBackward0>)
tensor(1.5586, grad_fn=<AddBackward0>)
tensor(1.2615, grad_fn=<AddBackward0>)
tensor(1.1694, grad_fn=<AddBackward0>)
tensor(1.0262, grad_fn=<AddBackward0>)
tensor(0.9069, grad_fn=<AddBackward0>)
tensor(0.8480, grad_fn=<AddBackward0>)
tensor(0.7911, grad_fn=<AddBackward0>)
tensor(0.7159, grad_fn=<AddBackward0>)
tensor(0.6842, grad_fn=<AddBackward0>)
tensor(0.6320, grad_fn=<AddBackward0>)
tensor(0.7325, grad_fn=<AddBackward0>)
tensor(0.7132, grad_fn=<AddBackward0>)
tensor(0.6225, grad_fn=<AddBackward0>)
tensor(0.5788, grad_fn=<AddBackward0>)
tensor(0.6615, grad_fn=<AddBackward0>)
tensor(0.5704, grad_fn=<AddBackward0>)
tensor(0.5413, grad_fn=<AddBackward0>)
tensor(0.5319, grad_fn=<AddBackward0>)
tensor(0.4991, grad_fn=<AddBackward0>)
tensor(0.5625, grad_fn=<AddBackward0>)
tensor(0.5188, grad_fn=<AddBackward0>)
tensor(0.4806, grad_fn=<AddBackward0>)
tensor(0.4630, grad_fn=<A

In [12]:
total_loss

tensor(0.2601, grad_fn=<AddBackward0>)

In [13]:
hidden # weight of 2nd hidden layer 

tensor([[ 0.2751,  0.2459, -0.3100,  0.5328, -0.1593, -0.0794,  0.4544, -0.0714,
         -0.4266,  0.4525,  0.5068,  0.5291,  0.3843, -0.4324,  0.1027, -0.2997,
         -0.2884, -0.4481,  0.7165,  0.7853, -0.7908,  0.3858,  0.5347,  0.0632,
          0.3540,  0.4212, -0.0365, -0.0933, -0.2715,  0.3714,  0.0446, -0.3503,
         -0.2726,  0.2046, -0.1102]], grad_fn=<TanhBackward>)

In [14]:
output.shape

torch.Size([1, 35])

In [15]:
start = torch.zeros(1, n_letters)
start[:,-2] = 1

with torch.no_grad():
    hidden = rnn.init_hidden()
    input_ = start
    output_string = ''
    for ii in range(len(my_string)):
        output, hidden = rnn.forward(input_, hidden)
        output_string += onehot2word(output.data)
        input_ = output
        
print(output_string)

hello  mncra    a n a r r r a a a a   a a a a a   a a a   a   a a a   a a   a                                    


In [16]:
rnn.forward(input_, hidden)

(tensor([[ 1.2289e-01, -3.4028e-02,  7.6688e-02,  6.6910e-03,  8.5520e-04,
           2.5233e-03,  1.1776e-02,  6.8985e-03,  5.6530e-02,  2.4144e-10,
           2.9340e-02,  4.8596e-02,  5.1241e-02,  1.0144e-01,  1.1097e-01,
          -2.1675e-02, -5.2606e-25,  2.3063e-03,  1.4199e-23,  1.8046e-03,
           4.8106e-02, -2.9513e-21,  6.8246e-03, -1.5174e-22,  3.3946e-02,
           4.5964e-23,  3.3706e-01,  1.2872e-02, -4.4398e-12,  8.8163e-03,
          -1.6006e-19,  1.3808e-17,  1.2806e-22, -7.9921e-22, -1.9913e-02]],
        grad_fn=<AddmmBackward>),
 tensor([[-6.2239e-02, -9.4547e-05,  1.1981e-01,  1.1001e-01, -7.0248e-02,
          -5.8333e-02, -1.3141e-01, -5.8123e-02,  6.2774e-02, -1.7042e-02,
           1.4157e-01, -2.0681e-01, -1.8637e-01, -1.8295e-02, -1.6850e-01,
          -2.8630e-02,  4.2617e-02,  1.9618e-02,  1.3478e-01, -1.9802e-01,
           1.0037e-02,  1.1015e-01, -1.1598e-01, -1.7958e-02, -4.8180e-02,
          -1.2899e-01,  1.8526e-01,  4.8830e-02,  3.2505e-02, -1

a를 출력하도록 saturated 된 것으로 보인다.

아주 간단히 구현한 것이고, 학습한 문자열도 길지 않아 엄청 좋은 결과를 얻기는 힘들다.

그저 torch로 rnn을 어떻게 구현하는지에 대해서 초점을 맞추자

hidden layer를 하나 늘려보기

In [28]:
class my_RNN_2(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(my_RNN_2, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.in2hid = nn.Linear(input_size, hidden_size)
        self.hid2hid = nn.Linear(hidden_size, hidden_size)
        self.hid2out = nn.Linear(hidden_size, output_size)
        self.act_fn = nn.Tanh()
        
    def forward(self, input, hidden):
        hidden = self.act_fn(self.in2hid(input)+self.hid2hid(hidden))
        
        hidden = self.act_fn(self.hid2hid(hidden)+self.hid2hid(hidden))
        output = self.hid2out(hidden)
        return output, hidden
    
    def init_hidden(self):
        return torch.zeros(1, self.hidden_size)
    

    

In [29]:
rnn_2 = my_RNN_2(n_letters, n_hidden, n_letters)

In [30]:
loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(rnn_2.parameters(), lr = lr)

In [31]:
for ii in range(epochs):
    rnn_2.zero_grad()
    total_loss = 0
    hidden = rnn_2.init_hidden()
    
    ## go all sequence and sum all loss
    for jj in range(one_hot.size()[0]-1):
        input_ = one_hot[jj:jj+1, :]
        target = one_hot[jj+1]
        
        output, hidden = rnn_2.forward(input_, hidden)
        loss = loss_func(output.view(-1), target.view(-1))
        
        total_loss += loss
        input_ = output
    
    # update weight after calculating all loss through sequence
    total_loss.backward()
    optimizer.step()
    
    if ii%10 == 0:
        print(total_loss)
    
    

tensor(6.7310, grad_fn=<AddBackward0>)
tensor(2.7471, grad_fn=<AddBackward0>)
tensor(2.2158, grad_fn=<AddBackward0>)
tensor(1.8703, grad_fn=<AddBackward0>)
tensor(1.6346, grad_fn=<AddBackward0>)
tensor(1.5235, grad_fn=<AddBackward0>)
tensor(1.4223, grad_fn=<AddBackward0>)
tensor(1.3180, grad_fn=<AddBackward0>)
tensor(1.1885, grad_fn=<AddBackward0>)
tensor(1.2760, grad_fn=<AddBackward0>)
tensor(1.0656, grad_fn=<AddBackward0>)
tensor(1.2328, grad_fn=<AddBackward0>)
tensor(1.0766, grad_fn=<AddBackward0>)
tensor(0.9415, grad_fn=<AddBackward0>)
tensor(0.9911, grad_fn=<AddBackward0>)
tensor(0.8892, grad_fn=<AddBackward0>)
tensor(0.8268, grad_fn=<AddBackward0>)
tensor(0.9249, grad_fn=<AddBackward0>)
tensor(0.8209, grad_fn=<AddBackward0>)
tensor(0.7585, grad_fn=<AddBackward0>)
tensor(0.8697, grad_fn=<AddBackward0>)
tensor(0.7923, grad_fn=<AddBackward0>)
tensor(0.7380, grad_fn=<AddBackward0>)
tensor(0.6808, grad_fn=<AddBackward0>)
tensor(0.7172, grad_fn=<AddBackward0>)
tensor(0.6528, grad_fn=<A

In [32]:
start = torch.zeros(1, n_letters)
start[:,-2] = 1

with torch.no_grad():
    hidden = rnn_2.init_hidden()
    input_ = start
    output_string = ''
    for ii in range(len(my_string)):
        output, hidden = rnn_2.forward(input_, hidden)
        output_string += onehot2word(output.data)
        input_ = output
        
print(output_string)

helle  en arenramenana oelcay arena eny  oym arank ayoyrarnea o   ay ako oyonrk a ee a rnoyo loarle cel o raeo ay


이전보다 saturation되는 경향은 덜하나, 여전히 어떤 말인지 알아보기는 힘들다.